In [1]:
import time
start = time.time()

import gurobipy as gb
import numpy as np
import csv

In [2]:
# Class which can have attributes set.
class expando(object):
    pass

In [3]:
#Adding Feasibility Cuts
class Lifted_ISSP:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.w_X_osrt = m.addVars([(b, c, d) for b in range(self.MP.data.S) for c in range(self.MP.data.R) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.w_I_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_I_ort1t")
        self.variables.Z_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Z_ort1t")
        self.variables.X_snt = m.addVars([(a, b, c) for a in range(self.MP.data.S) for b in range(self.MP.data.N) for c in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="X_snt")
        self.variables.Y_nrt1t = m.addVars([(a, b, c, d) for a in range(self.MP.data.N) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Y_nrt1t")
        self.variables.I_nt1t = m.addVars([(a, b, c) for a in range(self.MP.data.N) for b in range(self.MP.data.T) for c in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="I_nt1t")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        obj += sum(self.MP.data.w_C1*self.variables.w_I_ort1t[(r,t1,t)] for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau <= t - t1 < self.MP.data.gama)
        obj += sum(self.MP.data.h_C_sr[s][r]*self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S) for r in range(self.MP.data.R) for t in range(self.MP.data.T))     
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.w_X_osrt[(s,r,t)] for r in range(self.MP.data.R)) <= self.MP.data.w_beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T)), "c1")
        self.constraints.c2 = m.addConstrs((self.variables.w_I_ort1t[(r,t1,t)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1 == self.MP.data.tau), "c2")
        self.constraints.c3 = m.addConstrs((self.variables.w_I_ort1t[(r,t1,t)] - self.variables.w_I_ort1t[(r,t1,t-1)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t-t1 < self.MP.data.gama), "c3")
        self.constraints.c4 = m.addConstrs((sum(self.variables.Z_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T) if self.MP.data.tau <= t-t1 < self.MP.data.gama) + sum(self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S)) >= self.MP.data.demand[self.data.scenario][r][t] for r in range(self.MP.data.R) for t in range(self.MP.data.T)), "c4")
        self.constraints.c5 = m.addConstrs((sum(self.variables.w_I_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.w_alpha for r in range(self.MP.data.R) for t in range(self.MP.data.T)), "c5")
        self.constraints.c6 = m.addConstrs((sum(self.variables.X_snt[(s,n,t)] for n in range(self.MP.data.N)) <= self.MP.data.beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T)))
        self.constraints.c7 = m.addConstrs((sum(self.variables.X_snt[(s,n,t1)] for s in range(self.MP.data.S)) == self.variables.I_nt1t[(n,t1,t)] + sum(self.variables.Y_nrt1t[(n,r,t1,t)] for r in range(self.MP.data.R)) for n in range(self.MP.data.N) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1==self.MP.data.tau))
        self.constraints.c8 = m.addConstrs((self.variables.I_nt1t[(n,t1,t)] == self.variables.I_nt1t[(n,t1,t-1)] - sum(self.variables.Y_nrt1t[(n,r,t1,t)] for r in range(self.MP.data.R)) for n in range(self.MP.data.N) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t - t1 < self.MP.data.gama))
        self.constraints.c9 = m.addConstrs((sum(self.variables.I_nt1t[(n,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.alpha for n in range(self.MP.data.N) for t in range(self.MP.data.T)))
        

In [4]:
#Adding Feasibility Cuts
class Lifted_NISSP:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.w_X_osrt = m.addVars([(b, c, d) for b in range(self.MP.data.S) for c in range(self.MP.data.R) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.w_I_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_I_ort1t")
        self.variables.Z_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Z_ort1t")
        self.variables.X_snt = m.addVars([(a, b, c) for a in range(self.MP.data.S) for b in range(self.MP.data.N) for c in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="X_snt")
        self.variables.Y_nrt1t = m.addVars([(a, b, c, d) for a in range(self.MP.data.N) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Y_nrt1t")
        self.variables.I_nt1t = m.addVars([(a, b, c) for a in range(self.MP.data.N) for b in range(self.MP.data.T) for c in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="I_nt1t")
        self.variables.phi_ort = m.addVars([(b, c) for b in range(self.MP.data.R) for c in range(self.MP.data.T)], vtype=gb.GRB.CONTINUOUS, name="phi_ort")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        obj += sum(self.MP.data.w_C1*self.variables.w_I_ort1t[(r,t1,t)] for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau <= t - t1 < self.MP.data.gama)
        obj += sum(self.MP.data.h_C_sr[s][r]*self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S) for r in range(self.MP.data.R) for t in range(self.MP.data.T))     
        obj += sum(self.MP.data.delta_o*self.variables.phi_ort[(r,t)] for r in range(self.MP.data.R) for t in range(self.MP.data.T))
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs(sum(self.variables.w_X_osrt[(s,r,t)] for r in range(self.MP.data.R)) <= self.MP.data.w_beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T))
        self.constraints.c2 = m.addConstrs(self.variables.w_I_ort1t[(r,t1,t)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1 == self.MP.data.tau)
        self.constraints.c3 = m.addConstrs(self.variables.w_I_ort1t[(r,t1,t)] - self.variables.w_I_ort1t[(r,t1,t-1)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t-t1 < self.MP.data.gama)
        self.constraints.c4 = m.addConstrs(self.variables.phi_ort[(r,t)] + sum(self.variables.Z_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T) if self.MP.data.tau <= t-t1 < self.MP.data.gama) + sum(self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S)) >= self.MP.data.demand[self.data.scenario][r][t] for r in range(self.MP.data.R) for t in range(self.MP.data.T))
        self.constraints.c5 = m.addConstrs(sum(self.variables.w_I_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.w_alpha for r in range(self.MP.data.R) for t in range(self.MP.data.T))
        self.constraints.c6 = m.addConstrs((sum(self.variables.X_snt[(s,n,t)] for n in range(self.MP.data.N)) <= self.MP.data.beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T)))
        self.constraints.c7 = m.addConstrs((sum(self.variables.X_snt[(s,n,t1)] for s in range(self.MP.data.S)) == self.variables.I_nt1t[(n,t1,t)] + sum(self.variables.Y_nrt1t[(n,r,t1,t)] for r in range(self.MP.data.R)) for n in range(self.MP.data.N) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1==self.MP.data.tau))
        self.constraints.c8 = m.addConstrs((self.variables.I_nt1t[(n,t1,t)] == self.variables.I_nt1t[(n,t1,t-1)] - sum(self.variables.Y_nrt1t[(n,r,t1,t)] for r in range(self.MP.data.R)) for n in range(self.MP.data.N) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t - t1 < self.MP.data.gama))
        self.constraints.c9 = m.addConstrs((sum(self.variables.I_nt1t[(n,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.alpha for n in range(self.MP.data.N) for t in range(self.MP.data.T)))
        

In [5]:
# Master problem
class Relaxed_Master:
    def __init__(self,count,ep=0.000001, de=0.0001,scenarios=600):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.params = expando()
        self.params.scenarios = scenarios
        self.params.i = count
        self._load_data()
        self._build_model()
        self._init_benders_params(ep=ep, de=de)
        self._start_from_previous()
    
    def optimize(self):
        self.model.Params.lazyConstraints = 1
        #self.model.Params.Threads = 1
        start1 = time.time()
        h = np.zeros(self.data.O)
        
        for o in range(self.data.O):
               
            sm = Lifted_ISSP(self, o)
            sm.model.setParam ( "OutputFlag", 0 )
            sm.optimize()
            if(sm.model.status == gb.GRB.INFEASIBLE):
                
                self.model.addConstr(self.variables.beta_o[o] == 1)
                sm = Lifted_NISSP(self, o)
                sm.model.setParam ( "OutputFlag", 0 )
                sm.optimize()
                self.model.addConstr(self.variables.theta_o[o] >= sm.model.ObjVal)
            else:
                
                self.model.addConstr(self.variables.theta_o[o] >= sm.model.ObjVal*(1-self.variables.beta_o[o]))
            h[o] = sm.model.ObjVal   
        
        #eta = np.quantile(h, 0.5)
        #self.model.addConstr(self.variables.eta >= eta)
        
        self.model.update()  
        end1 = time.time()
        self.subs = {}
        self.submodels = {}
        self.feas_model = {}
        self.parameter_problems = {}
        self.opt_parameter_problems = {}
        for o in range(self.data.O):
            self.submodels[o] = {}
            self.submodels[o][0] = Subproblem_ISSPo(self,o)
            self.submodels[o][1] = Subproblem_NISSPo(self,o)
            self.parameter_problems[o] = parameter_problemo(self,o)
            self.opt_parameter_problems[o] = opt_parameter_problemo(self,o)
            
        print("Time spent to pre-process the code: " + str(end1-start1), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))    
        self.model._vars = self.model.getVars()
        #self.model.Params.TimeLimit = 1000
        self.model.Params.LogToConsole = 0
        self.model.Params.LogFile = "sol16-800-"+str(self.params.i+1)
        start = time.time()
        self.model.optimize(callbackfunc)
        end = time.time()
        print ("Time taken by the code for the instance " + str(self.params.i+1) + "= " + str(end-start), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))
        print ("Time taken by the extra linearization in the code " + "= " + str(self.data.extra), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))  
        
        print ("Upper Bound: "+str(self.data.ub), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))  
        print ("Lower Bound: "+str(self.data.lb), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))
        print ("MIP Gap:" + str((((m.data.ub-m.data.lb)*100)/m.data.lb)), file=open("output16-800-"+str(self.params.i+1)+".txt", "a"))

            
    def _init_benders_params(self, ep=0.000001, de=0.000001):
        self.data.upper_bounds = []
        self.data.lower_bounds = []
        self.data.ub = gb.GRB.INFINITY
        self.data.lb = -gb.GRB.INFINITY 
        self.data.Theta = 0
        self.data.V = 0
        self.data.Zi = 0
        self.data.Z_eta = 0
        self.data.phi_ort = np.zeros((self.data.O, self.data.R, self.data.T))
    
    def _load_data(self):
        self.data.S = 4
        self.data.N = 6
        self.data.R = 10
        self.data.T = 6
        self.data.O = self.params.scenarios
        self.data.X_snt = np.zeros((self.data.N, self.data.R, self.data.T, self.data.T))
        self.data.Y_nrt1t = np.zeros((self.data.N, self.data.R, self.data.T, self.data.T))
        self.data.I_nt1t = np.zeros((self.data.N, self.data.R, self.data.T, self.data.T))
        self.data.eta = 0
        self.data.extra = 0
        fil_name = 'beta_st20'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.beta_st = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.beta_st.append(nwrow)
        """    
        fil_name = 'w_beta_st20'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.w_beta_st = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.w_beta_st.append(nwrow)
        """
        self.data.w_beta_st = np.zeros((self.data.S,self.data.T))
        for s in range(self.data.S):
            for t in range(self.data.T):
                self.data.w_beta_st[s][t] = 36500
                
        fil_name = 'C_sn20'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.C_sn = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.C_sn.append(nwrow)
            
        fil_name = 'w_C_nr20'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.w_C_nr = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.w_C_nr.append(nwrow)
            
        fil_name = 'h_C_sr20'
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.h_C_sr = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.h_C_sr.append(nwrow)            
            
        fil_name = 'demand_data20-'+str(self.params.i+1)
        with open(fil_name+'.csv', 'r') as f:
            reader = csv.reader(f)
            examples = list(reader)
        self.data.demand = []
        for row in examples:
            nwrow = []
            for r in row:
                nwrow.append(eval(r))
            self.data.demand.append(nwrow)
        
        for o in range(self.data.O):
            for r in range(self.data.R):
                self.data.demand[o][r][0] = 0
        
        
        self.data.C1 = 250
        self.data.w_C1 = 400
        self.data.tau = 1
        self.data.gama = 4
        self.data.alpha = 60000
        self.data.w_alpha = 10000
        self.data.p_o = 1/self.data.O
        self.data.delta_o = 9500
        self.data.epsilon = 0.1
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()  
        
    def _build_variables(self):
        m = self.model
        self.variables.X_snt = m.addVars([(a, b, c) for a in range(self.data.S) for b in range(self.data.N) for c in range(self.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="X_snt")
        self.variables.Y_nrt1t = m.addVars([(a, b, c, d) for a in range(self.data.N) for b in range(self.data.R) for c in range(self.data.T) for d in range(self.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Y_nrt1t")
        self.variables.I_nt1t = m.addVars([(a, b, c) for a in range(self.data.N) for b in range(self.data.T) for c in range(self.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="I_nt1t")
        self.variables.beta_o = m.addVars([(a) for a in range(self.data.O)], vtype=gb.GRB.BINARY, name="beta_o")
        self.variables.v = m.addVars([(a) for a in range(self.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="v_o")
        self.variables.eta = m.addVar(lb = 0, vtype=gb.GRB.CONTINUOUS, name="eta")
        self.variables.theta_o = m.addVars([(a) for a in range(self.data.O)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="theta_o") 
        self.variables.zeta = m.addVar(lb = 0, vtype=gb.GRB.CONTINUOUS, name="zeta")
        m.update()
        
    def _build_objective(self):
        obj = 0
        obj += sum(sum(self.variables.X_snt[(s, n, t)] for t in range(self.data.T))*self.data.C_sn[s][n] for s in range(self.data.S) for n in range(self.data.N))
        obj += sum(self.data.C1*self.variables.I_nt1t[(n,t1,t)] for n in range(self.data.N) for t1 in range(self.data.T) for t in range(self.data.T) if self.data.tau <= t - t1 < self.data.gama)
        obj += sum(self.variables.Y_nrt1t[(n,r,t1,t)]*self.data.w_C_nr[n][r] for n in range(self.data.N) for r in range(self.data.R) for t in range(self.data.T) for t1 in range(self.data.T))
        obj += 0.9*self.variables.zeta
        obj += 0.1*(self.variables.eta + (1/(1-0.9))*sum(self.data.p_o*self.variables.v[o] for o in range(self.data.O)))
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.X_snt[(s,n,t)] for n in range(self.data.N)) <= self.data.beta_st[s][t] for s in range(self.data.S) for t in range(self.data.T)))
        self.constraints.c2 = m.addConstrs((sum(self.variables.X_snt[(s,n,t1)] for s in range(self.data.S)) == self.variables.I_nt1t[(n,t1,t)] + sum(self.variables.Y_nrt1t[(n,r,t1,t)] for r in range(self.data.R)) for n in range(self.data.N) for t1 in range(self.data.T) for t in range(self.data.T) if t-t1==self.data.tau))
        self.constraints.c3 = m.addConstrs((self.variables.I_nt1t[(n,t1,t)] == self.variables.I_nt1t[(n,t1,t-1)] - sum(self.variables.Y_nrt1t[(n,r,t1,t)] for r in range(self.data.R)) for n in range(self.data.N) for t1 in range(self.data.T) for t in range(self.data.T) if self.data.tau < t - t1 < self.data.gama))
        self.constraints.c4 = m.addConstrs((sum(self.variables.I_nt1t[(n,t1,t)] for t1 in range(self.data.T)) <= self.data.alpha for n in range(self.data.N) for t in range(self.data.T)))
        self.constraints.c5 = m.addConstr((sum(self.data.p_o*self.variables.beta_o[o] for o in range(self.data.O)) <= self.data.epsilon))    
        self.constraints.c6 = m.addConstrs((self.variables.v[o] >= self.variables.theta_o[o] - self.variables.eta for o in range(self.data.O)))
        self.constraints.c7 = m.addConstrs((self.variables.zeta >= self.variables.theta_o[o] for o in range(self.data.O)))
        pass
        
    def _start_from_previous(self):
        '''
            Used to warm-start MIP problems.
        '''
        pass


In [6]:
class opt_parameter_problemo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.w_X_osrt = m.addVars([(b, c, d) for b in range(self.MP.data.S) for c in range(self.MP.data.R) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.w_I_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_I_ort1t")
        self.variables.Z_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Z_ort1t")
        self.variables.phi_ort = m.addVars([(b, c) for b in range(self.MP.data.R) for c in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="phi_ort")
        self.variables.Y_nrt1t = m.addVars([(a,b,c,d) for a in range(self.MP.data.N) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name = "Y_nrt1t_fix")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs(sum(self.variables.w_X_osrt[(s,r,t)] for r in range(self.MP.data.R)) <= self.MP.data.w_beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T))
        self.constraints.c2 = m.addConstrs(self.variables.w_I_ort1t[(r,t1,t)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1 == self.MP.data.tau)
        self.constraints.c3 = m.addConstrs(self.variables.w_I_ort1t[(r,t1,t)] - self.variables.w_I_ort1t[(r,t1,t-1)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t-t1 < self.MP.data.gama)
        self.constraints.c4 = m.addConstrs(self.variables.phi_ort[(r,t)] + sum(self.variables.Z_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T) if self.MP.data.tau <= t-t1 < self.MP.data.gama) + sum(self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S)) >= self.MP.data.demand[self.data.scenario][r][t] for r in range(self.MP.data.R) for t in range(self.MP.data.T))
        self.constraints.c5 = m.addConstrs(sum(self.variables.w_I_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.w_alpha for r in range(self.MP.data.R) for t in range(self.MP.data.T))
           

In [7]:
class parameter_problemo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.w_X_osrt = m.addVars([(b, c, d) for b in range(self.MP.data.S) for c in range(self.MP.data.R) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.w_I_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_I_ort1t")
        self.variables.Z_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Z_ort1t")        
        self.variables.Y_nrt1t = m.addVars([(a, b, c, d) for a in range(self.MP.data.N) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Y_nrt1t")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.w_X_osrt[(s,r,t)] for r in range(self.MP.data.R)) <= self.MP.data.w_beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T)), "c1")
        self.constraints.c2 = m.addConstrs((self.variables.w_I_ort1t[(r,t1,t)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1 == self.MP.data.tau), "c2")
        self.constraints.c3 = m.addConstrs((self.variables.w_I_ort1t[(r,t1,t)] - self.variables.w_I_ort1t[(r,t1,t-1)] + self.variables.Z_ort1t[(r,t1,t)] == sum(self.variables.Y_nrt1t[(n,r,t1,t)] for n in range(self.MP.data.N)) for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t-t1 < self.MP.data.gama), "c3")
        self.constraints.c4 = m.addConstrs((sum(self.variables.Z_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T) if self.MP.data.tau <= t-t1 < self.MP.data.gama) + sum(self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S)) >= self.MP.data.demand[self.data.scenario][r][t] for r in range(self.MP.data.R) for t in range(self.MP.data.T)), "c4")
        self.constraints.c5 = m.addConstrs((sum(self.variables.w_I_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.w_alpha for r in range(self.MP.data.R) for t in range(self.MP.data.T)), "c5")
        

In [8]:
class Subproblem_ISSPo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()
      

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.w_X_osrt = m.addVars([(b, c, d) for b in range(self.MP.data.S) for c in range(self.MP.data.R) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.w_I_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="w_I_ort1t")
        self.variables.Z_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], lb = 0, vtype=gb.GRB.CONTINUOUS, name="Z_ort1t")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
        
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs((sum(self.variables.w_X_osrt[(s,r,t)] for r in range(self.MP.data.R)) <= self.MP.data.w_beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T)), "c1")
        self.constraints.c2 = m.addConstrs((self.variables.w_I_ort1t[(r,t1,t)] + self.variables.Z_ort1t[(r,t1,t)] == 0 for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1 == self.MP.data.tau), "c2")
        self.constraints.c3 = m.addConstrs((self.variables.w_I_ort1t[(r,t1,t)] - self.variables.w_I_ort1t[(r,t1,t-1)] + self.variables.Z_ort1t[(r,t1,t)] == 0 for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t-t1 < self.MP.data.gama), "c3")
        self.constraints.c4 = m.addConstrs((sum(self.variables.Z_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T) if self.MP.data.tau <= t-t1 < self.MP.data.gama) + sum(self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S)) >= self.MP.data.demand[self.data.scenario][r][t] for r in range(self.MP.data.R) for t in range(self.MP.data.T)), "c4")
        self.constraints.c5 = m.addConstrs((sum(self.variables.w_I_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.w_alpha for r in range(self.MP.data.R) for t in range(self.MP.data.T)), "c5")
        

In [9]:
# Subproblem-2
class Subproblem_NISSPo:
    def __init__(self, MP, scenario):
        self.data = expando()
        self.variables = expando()
        self.constraints = expando()
        self.results = expando()
        self.MP = MP
        self.data.scenario = scenario
        self._build_model()

    def optimize(self):
        self.model.optimize()
        
    def _build_model(self):
        self.model = gb.Model()
        self._build_variables()
        self._build_objective()
        self._build_constraints()
        self.model.update()
        
    def _build_variables(self):
        m = self.model
        self.variables.w_X_osrt = m.addVars([(b, c, d) for b in range(self.MP.data.S) for c in range(self.MP.data.R) for d in range(self.MP.data.T)], vtype=gb.GRB.CONTINUOUS, name="w_X_osrt")
        self.variables.w_I_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], vtype=gb.GRB.CONTINUOUS, name="w_I_ort1t")
        self.variables.Z_ort1t = m.addVars([(b, c, d) for b in range(self.MP.data.R) for c in range(self.MP.data.T) for d in range(self.MP.data.T)], vtype=gb.GRB.CONTINUOUS, name="Z_ort1t")
        self.variables.phi_ort = m.addVars([(b, c) for b in range(self.MP.data.R) for c in range(self.MP.data.T)], vtype=gb.GRB.CONTINUOUS, name="phi_ort")
        m.update()
        
    def _build_objective(self):
        m = self.model
        obj = 0
        self.model.setObjective(obj, gb.GRB.MINIMIZE)
    
    def _build_constraints(self):
        m = self.model
        self.constraints.c1 = m.addConstrs(sum(self.variables.w_X_osrt[(s,r,t)] for r in range(self.MP.data.R)) <= self.MP.data.w_beta_st[s][t] for s in range(self.MP.data.S) for t in range(self.MP.data.T))
        self.constraints.c2 = m.addConstrs(self.variables.w_I_ort1t[(r,t1,t)] + self.variables.Z_ort1t[(r,t1,t)] == 0 for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if t-t1 == self.MP.data.tau)
        self.constraints.c3 = m.addConstrs(self.variables.w_I_ort1t[(r,t1,t)] - self.variables.w_I_ort1t[(r,t1,t-1)] + self.variables.Z_ort1t[(r,t1,t)] == 0 for r in range(self.MP.data.R) for t1 in range(self.MP.data.T) for t in range(self.MP.data.T) if self.MP.data.tau < t-t1 < self.MP.data.gama)
        self.constraints.c4 = m.addConstrs(self.variables.phi_ort[(r,t)] + sum(self.variables.Z_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T) if self.MP.data.tau <= t-t1 < self.MP.data.gama) + sum(self.variables.w_X_osrt[(s,r,t)] for s in range(self.MP.data.S)) >= self.MP.data.demand[self.data.scenario][r][t] for r in range(self.MP.data.R) for t in range(self.MP.data.T))
        self.constraints.c5 = m.addConstrs(sum(self.variables.w_I_ort1t[(r,t1,t)] for t1 in range(self.MP.data.T)) <= self.MP.data.w_alpha for r in range(self.MP.data.R) for t in range(self.MP.data.T))
        

In [10]:
def callbackfunc(model, where):      
    
    if where == gb.GRB.Callback.MIPSOL:
        Feas_flag = False
        Cut_found = False
        vList = model.cbGetSolution(model.getVars())
        
        
        z = model.cbGet(gb.GRB.Callback.MIPSOL_OBJ)
        #print ("First Variables")
        
        X_snt = np.zeros((m.data.S, m.data.N, m.data.T))
        I_nt1t = np.zeros((m.data.N, m.data.T, m.data.T))
        Y_nrt1t = np.zeros((m.data.N, m.data.R, m.data.T, m.data.T))                
        eta = vList[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + m.data.O + m.data.O]
        zeta = vList[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + m.data.O + m.data.O + 1 + m.data.O]
        theta =np.zeros(m.data.O)
        beta_o = np.zeros(m.data.O)
        v = np.zeros(m.data.O)
        for o in range(m.data.O):
            theta[o] = vList[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + m.data.O + m.data.O + 1 + o]
            v[o] = vList[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + m.data.O + o]
            beta_o[o] = vList[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + o]
        for n in range(m.data.N):
            for t in range(m.data.T):
                for t1 in range(m.data.T):
                    I_nt1t[n][t1][t] = vList[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + n*m.data.T*m.data.T + t1*m.data.T + t]
                    for r in range(m.data.R):
                        Y_nrt1t[n][r][t1][t] = vList[m.data.S*m.data.N*m.data.T + n*m.data.R*m.data.T*m.data.T + r*m.data.T*m.data.T + t1*m.data.T + t]
                for s in range(m.data.S):
                    X_snt[s][n][t] = vList[s*m.data.N*m.data.T + n*m.data.T + t]
        
        sub = np.zeros(m.data.O) 
        
        
        for scenario in range(m.params.scenarios):
            
            if (Feas_flag == True):
                break
                
            if (beta_o[scenario] == 0):
                start2 = time.time()
                sm = m.submodels[scenario][0]
                #sm.model.setParam ( "OutputFlag", 0 )
                for r in range(m.data.R):
                    for t1 in range(m.data.T):
                        for t in range(m.data.T):
                            if (t - t1 == m.data.tau):
                                sm.constraints.c2[(r,t1,t)].rhs = sum(Y_nrt1t[n][r][t1][t] for n in range(m.data.N))
                            if (m.data.tau<t-t1<m.data.gama):
                                sm.constraints.c3[(r,t1,t)].rhs = sum(Y_nrt1t[n][r][t1][t] for n in range(m.data.N))
                
                obj = sum(m.data.w_C1*sm.variables.w_I_ort1t[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T) if m.data.tau <= t - t1 < m.data.gama)
                obj += sum(m.data.h_C_sr[s][r]*sm.variables.w_X_osrt[(s,r,t)] for s in range(m.data.S) for r in range(m.data.R) for t in range(m.data.T))     
                sm.model.setObjective(obj, gb.GRB.MINIMIZE) 
                sm.model.setParam ( "OutputFlag", 0 )
                sm.model.setParam ( "InfUnbdInfo", 1 )
                end2 = time.time()
                m.data.extra += (end2-start2)
                sm.model.optimize()
                    
                    
                if (sm.model.status == gb.GRB.INFEASIBLE):
                    #start2 = time.time()
                    #sm = m.feas_model[scenario]
                    #for n in range(m.data.N):
                    #    for r in range(m.data.R):
                    #        for t1 in range(m.data.T):
                    #            for t in range(m.data.T):
                    #                sm.constraints.fix_y1[(n,r,t1,t)].rhs = Y_nrt1t[n][r][t1][t]
                    #obj = sum(m.data.w_C1*sm.variables.w_I_ort1t[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T) if m.data.tau <= t - t1 < m.data.gama)
                    #obj += sum(m.data.h_C_sr[s][r]*sm.variables.w_X_osrt[(s,r,t)] for s in range(m.data.S) for r in range(m.data.R) for t in range(m.data.T))     
                    #sm.model.setObjective(obj, gb.GRB.MINIMIZE) 
                    
                    
                    #end2 = time.time()
                    #m.data.extra += (end2-start2)
                    #sm.model.optimize()
                    
                    
                    lambda2 = np.zeros((m.data.R, m.data.T, m.data.T))
                    lambda3 = np.zeros((m.data.R, m.data.T, m.data.T))
                    #lambda6 = np.zeros((m.data.N, m.data.R, m.data.T, m.data.T))
                        
                    for t in range(m.data.T):
                        for r in range(m.data.R):
                            for t1 in range(m.data.T):
                                if (t - t1 == m.data.tau):
                                    lambda2[(r,t1,t)] = sm.constraints.c2[(r,t1,t)].FarkasDual
                                if (m.data.tau<t-t1<m.data.gama):
                                    lambda3[(r,t1,t)] = sm.constraints.c3[(r,t1,t)].FarkasDual
                                #for n in range(m.data.N):
                                    #lambda6[(n,r,t1,t)] = sm.constraints.fix_y1[(n,r,t1,t)].FarkasDual                                 
                                
                    h_val = {}    
                    
                    for o in range(m.data.O):
                        start2 = time.time()
                        #sm1 = m.parameter_problems[o]
                        obj = sum(sum(-m.parameter_problems[o].variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*-1*lambda2[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))
                        obj += sum(sum(-m.parameter_problems[o].variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*-1*lambda3[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))
                        #obj += sum(-1*lambda6[(n,r,t1,t)]*(-sm1.variables.Y_nrt1t[(n,r,t1,t)]) for n in range(m.data.N) for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))
                        m.parameter_problems[o].model.setObjective(obj, gb.GRB.MINIMIZE)
                        m.parameter_problems[o].model.setParam ("OutputFlag", 0) 
                        end2 = time.time()
                        m.data.extra += (end2-start2)
                        m.parameter_problems[o].model.optimize()
                        h_val[o] = m.parameter_problems[o].model.ObjVal
                    h_zeta = {k: v for k, v in sorted(h_val.items(), key=lambda item: item[1], reverse=True)}
                    keys = list(h_zeta.keys())
                    
                    if (beta_o[keys[0]] == 0):
                        m.model.cbLazy(sum(sum(-model._vars[m.data.S*m.data.N*m.data.T + n*m.data.R*m.data.T*m.data.T + r*m.data.T*m.data.T + t1*m.data.T + t] for n in range(m.data.N))*-1*lambda2[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T)) +
                                                sum(sum(-model._vars[m.data.S*m.data.N*m.data.T + n*m.data.R*m.data.T*m.data.T + r*m.data.T*m.data.T + t1*m.data.T + t] for n in range(m.data.N))*-1*lambda3[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T)) +
                                                #sum(model._vars[-m.data.S*m.data.N*m.data.T + n*m.data.R*m.data.T*m.data.T + r*m.data.T*m.data.T + t1*m.data.T + t]*-1*lambda6[(n,r,t1,t)] for n in range(m.data.N) for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T)) +
                                                (h_val[keys[0]] - h_val[keys[int((m.data.epsilon)*m.data.O)]])*model._vars[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + keys[0]] >= h_val[keys[0]])
                        
                        
                    else:
                        for i in range(int((m.data.epsilon)*m.data.O)+1):
                            if (beta_o[keys[i+1]] == 0):
                                s = i+1
                                break
                        m.model.cbLazy(sum(sum(-m.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*-1*lambda2[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T)) +
                                                sum(sum(-m.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*-1*lambda3[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T)) +
                                                #sum(model._vars[m.data.S*m.data.N*m.data.T + n*m.data.R*m.data.T*m.data.T + r*m.data.T*m.data.T + t1*m.data.T + t]*-1*lambda6[(n,r,t1,t)] for n in range(m.data.N) for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T)) +
                                                (h_val[keys[0]] - h_val[keys[s]])*(model._vars[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + keys[0]]) +
                                                (h_val[keys[s]] - h_val[keys[int((m.data.epsilon)*m.data.O)]])*(model._vars[m.data.S*m.data.N*m.data.T + m.data.N*m.data.R*m.data.T*m.data.T + m.data.N*m.data.T*m.data.T + keys[s]]) >= h_val[keys[0]])
                        
                            
                        
                    Feas_flag = True

                    
                    
                else:
                    sub[scenario] = sm.model.ObjVal
                    pi1 = np.zeros((m.data.S, m.data.T))
                    pi2 = np.zeros((m.data.R, m.data.T, m.data.T))
                    pi3 = np.zeros((m.data.R, m.data.T, m.data.T))
                    pi4 = np.zeros((m.data.R, m.data.T))
                    pi5 = np.zeros((m.data.R, m.data.T))
                        
                    for t in range(m.data.T):
                        for s in range(m.data.S):
                            pi1[(s,t)] = sm.constraints.c1[(s,t)].pi
                        for r in range(m.data.R):
                            for t1 in range(m.data.T):
                                if (t - t1 == m.data.tau):
                                    pi2[(r,t1,t)] = sm.constraints.c2[(r,t1,t)].pi
                                if (m.data.tau<t-t1<m.data.gama):
                                    pi3[(r,t1,t)] = sm.constraints.c3[(r,t1,t)].pi
                                
                            pi4[(r,t)] = sm.constraints.c4[(r,t)].pi
                            pi5[(r,t)] = sm.constraints.c5[(r,t)].pi  
                    start2 = time.time()
                    vi = m.opt_parameter_problems[scenario]
                    obj = (sum(sum(-vi.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*pi2[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))) 
                    obj += (sum(sum(-vi.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*pi3[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))) 
                    obj += sum(m.data.w_C1*vi.variables.w_I_ort1t[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T) if m.data.tau <= t - t1 < m.data.gama)
                    obj += sum(m.data.h_C_sr[s][r]*vi.variables.w_X_osrt[(s,r,t)] for s in range(m.data.S) for r in range(m.data.R) for t in range(m.data.T))     
                    obj += sum(m.data.delta_o*vi.variables.phi_ort[(r,t)] for r in range(m.data.R) for t in range(m.data.T))
                    vi.model.setObjective(obj, gb.GRB.MINIMIZE)
                    vi.model.setParam ( "OutputFlag", 0 )    
                    end2 = time.time()
                    m.data.extra += (end2-start2)
                    vi.model.optimize()
                    h_v = vi.model.ObjVal
                    
                    if (sub[scenario] > theta[scenario]):
                        m.model.cbLazy(m.variables.theta_o[scenario] >= sum(pi1[(s,t)]*(1 - m.variables.beta_o[scenario])*m.data.w_beta_st[s][t] for s in range(m.data.S) for t in range(m.data.T)) +
                                            (sum(sum(m.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*pi2[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))) +
                                            (sum(sum(m.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*pi3[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))) +
                                            (sum(m.data.demand[scenario][r][t]*pi4[(r,t)]*(1 - m.variables.beta_o[scenario]) for r in range(m.data.R) for t in range(m.data.T))) +
                                            (sum(m.data.w_alpha*pi5[(r,t)]*(1 - m.variables.beta_o[scenario]) for r in range(m.data.R) for t in range(m.data.T))) +
                                            (h_v*m.variables.beta_o[scenario]))  
                        
                    Cut_found = True
                     
            else: 
                start2 = time.time()
                sm = m.submodels[scenario][1]
                for r in range(m.data.R):
                    for t1 in range(m.data.T):
                        for t in range(m.data.T):
                            if (t - t1 == m.data.tau):
                                sm.constraints.c2[(r,t1,t)].rhs = sum(Y_nrt1t[n][r][t1][t] for n in range(m.data.N))
                            if (m.data.tau<t-t1<m.data.gama):
                                sm.constraints.c3[(r,t1,t)].rhs = sum(Y_nrt1t[n][r][t1][t] for n in range(m.data.N))
                obj = sum(m.data.w_C1*sm.variables.w_I_ort1t[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T) if m.data.tau <= t - t1 < m.data.gama)
                obj += sum(m.data.h_C_sr[s][r]*sm.variables.w_X_osrt[(s,r,t)] for s in range(m.data.S) for r in range(m.data.R) for t in range(m.data.T))     
                obj += sum(m.data.delta_o*sm.variables.phi_ort[(r,t)] for r in range(m.data.R) for t in range(m.data.T))
                sm.model.setObjective(obj, gb.GRB.MINIMIZE) 
                sm.model.setParam ( "OutputFlag", 0 )
                end2 = time.time()
                m.data.extra += (end2-start2)
                sm.model.optimize()
                sub[scenario] = sm.model.ObjVal    
                    
                pi1 = np.zeros((m.data.S, m.data.T))
                pi2 = np.zeros((m.data.R, m.data.T, m.data.T))
                pi3 = np.zeros((m.data.R, m.data.T, m.data.T))
                pi4 = np.zeros((m.data.R, m.data.T))
                pi5 = np.zeros((m.data.R, m.data.T))
        
                for t in range(m.data.T):
                    for s in range(m.data.S):
                        pi1[(s,t)] = sm.constraints.c1[(s,t)].pi
                    for r in range(m.data.R):
                        for t1 in range(m.data.T):
                            if (t - t1 == m.data.tau):
                                pi2[(r,t1,t)] = sm.constraints.c2[(r,t1,t)].pi
                            if (m.data.tau<t-t1<m.data.gama):
                                pi3[(r,t1,t)] = sm.constraints.c3[(r,t1,t)].pi
                            
                        pi4[(r,t)] = sm.constraints.c4[(r,t)].pi
                        pi5[(r,t)] = sm.constraints.c5[(r,t)].pi                 
                    
                if (sub[scenario] > theta[scenario]):
                    m.model.cbLazy(m.variables.theta_o[scenario] >= 
                                  (sum(pi1[(s,t)]*m.data.w_beta_st[s][t] for s in range(m.data.S) for t in range(m.data.T)) +
                                        (sum(sum(m.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*pi2[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))) +
                                        (sum(sum(m.variables.Y_nrt1t[(n,r,t1,t)] for n in range(m.data.N))*pi3[(r,t1,t)] for r in range(m.data.R) for t1 in range(m.data.T) for t in range(m.data.T))) +
                                        (sum(m.data.demand[scenario][r][t]*pi4[(r,t)] for r in range(m.data.R) for t in range(m.data.T))) +
                                        (sum(m.data.w_alpha*pi5[(r,t)] for r in range(m.data.R) for t in range(m.data.T)))))
                    
                Cut_found = True
                    
            
        
        
        
        if (True):
            
            if (Feas_flag == False):

                m.data.lb = z
                Ze = 0
                Zi = 0
                Z_eta = 0
                V = 0
                for s in range(m.data.O):
                    if (Ze < sub[s]):
                        Ze = sub[s]
                    V += m.data.p_o*(1/(1-0.9))*v[s]
                    Z_eta += m.data.p_o*(1/(1-0.9))*max((sub[s] - eta),0)
            
                temp = z - 0.9*zeta + 0.9*Ze - 0.1*V + 0.1*Z_eta
                if (m.data.ub >= temp):
                    m.data.ub = temp
                if (m.data.ub - m.data.lb < 0.000001*m.data.lb):
                    m.model.terminate()

            
            
    

In [ ]:
for i in range(5):
    m = Relaxed_Master(i)
    m.optimize()
"""print ("C1:" + str(sum(sum(m.variables.X_snt[(s, n, t)].x for t in range(m.data.T))*m.data.C_sn[s][n] for s in range(m.data.S) for n in range(m.data.N))), file=open("output20-100-"+str(i+1)+".txt", "a"))
    print ("C2:" + str(sum(m.data.C1*m.variables.I_nt1t[(n,t1,t)].x for n in range(m.data.N) for t1 in range(m.data.T) for t in range(m.data.T) if m.data.tau <= t - t1 < m.data.gama)), file=open("output20-100-"+str(i+1)+".txt", "a"))
    print ("C3:" + str(sum(m.variables.Y_nrt1t[(n,r,t1,t)].x*m.data.w_C_nr[n][r] for n in range(m.data.N) for r in range(m.data.R) for t in range(m.data.T) for t1 in range(m.data.T))), file=open("output20-100-"+str(i+1)+".txt", "a"))
    print ("C5:" + str(0.5*(m.variables.eta.x + (1/(1-0.9))*sum(m.data.p_o*max(m.subs[o].model.ObjVal-m.variables.eta.x, 0) for o in range(m.data.O)))), file=open("output20-100-"+str(i+1)+".txt", "a"))
    print ("C4:" + str(0.5*(sum(m.data.p_o*m.subs[o].model.ObjVal for o in range(m.data.O)))), file=open("output20-100-"+str(i+1)+".txt", "a"))
    for o in range(m.data.O):
        if (m.variables.beta_o[o].x != 0):
            for r in range(m.data.R):
                for t in range(m.data.T):
                    if (m.data.phi_ort[o][r][t] != 0):
                        print ("Shortage at retailer " + str(r) + " in time period " + str(t) + " is: " + str(m.data.phi_ort[o][r][t]), file=open("output20-100-"+str(i+1)+".txt", "a"))"""

Using license file /software/commercial/gurobi/gurobi.lic
Set parameter TokenServer to value license8.clemson.edu
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
